# Segmenting and Clustering Neighborhoods in Toronto

Downloading Dependencies

In [6]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Importing BeautifulSoup

In [7]:
from bs4 import BeautifulSoup

Import get function from requests. Assigning the address of the web page to a variable named url.
Requesting the server the content of the web page by using get(), and storing the server’s response in the variable response.

In [8]:
from requests import get
url= "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = get(url)
print(response.text[:500])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","w


Reading table from wiki page using BeautifulSoup

In [9]:
html_wiki_page= BeautifulSoup(response.text, 'html.parser')
type(html_wiki_page)
page_tables = html_wiki_page.find_all('table')
Toronto_FSAs_table=page_tables[0]
Toronto_FSAs_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

Creating Dataframe
- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
.

In [10]:
#del Toronto_FSAs_df

In [11]:

n_columns=0
n_rows=0
column_names=[]

for row in Toronto_FSAs_table.find_all('tr'):
    td_tags=row.find_all('td')
    if len(td_tags)>0:
        n_rows+=1
        if n_columns == 0:
            n_columns=len(td_tags)
    
    th_tags=row.find_all('th')
    if len(th_tags) > 0 and len(column_names) == 0:
        for th in th_tags:
            column_names.append(th.get_text().rstrip())
            


Toronto_FSAs_df= pd.DataFrame(columns=column_names,index=range(0,n_rows))



row_marker=-1

for row in Toronto_FSAs_table.find_all('tr'):
    column_marker=0
    columns=row.find_all('td')
    for column in columns:
        #print('row{},column{},text()'.format(row_marker,column_marker,column.get_text()))
        Toronto_FSAs_df.loc[row_marker, column_names[column_marker]] = column.get_text().rstrip()
        column_marker+=1
    row_marker+=1
    
Toronto_FSAs_df.head()  

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [12]:
Toronto_FSAs_df=Toronto_FSAs_df[Toronto_FSAs_df.Borough != 'Not assigned']
Toronto_FSAs_df.reset_index()
Toronto_FSAs_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
Toronto_FSAs_df.loc[Toronto_FSAs_df['Neighbourhood']=='Not assigned', 'Neighbourhood']=Toronto_FSAs_df['Borough']
Toronto_FSAs_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [14]:
Toronto_FSAs_df_grouped=Toronto_FSAs_df.groupby(['Postcode','Borough'],as_index=False)['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame(name='Neighbourhood').reset_index()

Toronto_FSAs_df_grouped.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [15]:
Toronto_FSAs_df_grouped.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [16]:
import geocoder

### Following is the code to get lat long from geocoder, but not able to get the geographical coordinates of the neighborhoods using the Geocoder package

for index, row in Toronto_FSAs_df_grouped.iterrows():
   
    postal_code=row['Postcode']
#initialize your variable to None
    lat_lng_coords = None

#loop until you get the coordinates

    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
#Assign Latitude and longitude
    row['Latitude']= lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]


Toronto_FSAs_df_grouped.head()

In [17]:
Geospatial_data=pd.read_csv("https://cocl.us/Geospatial_data")

In [18]:
Geospatial_data.columns=['Postcode', 'Latitude', 'Longitude']
Geospatial_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
Toronto_FSAs=pd.merge(Toronto_FSAs_df_grouped,Geospatial_data,on='Postcode')
Toronto_FSAs.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Generate maps to visualize your neighborhoods

In [27]:
import folium
latitude =Toronto_FSAs.Latitude.mean()
longitude =Toronto_FSAs.Longitude.mean()
venues_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around M1B


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(Toronto_FSAs.Latitude, Toronto_FSAs.Longitude, Toronto_FSAs.Postcode):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

## Explore a location

#### Define URL and client codes

In [31]:
CLIENT_ID="RFBWD0MXEMM3ESEHAPAJMAPGOA4GF5NBWA15BU1CYSRLLWSZ"
CLIENT_SECRET="M3LQZF1E0YG0EQJ0IUMDUFDQSGZN0RHNFXDH335O44CYPAIR"
VERSION='20180604'
radius=500
LIMIT=30



url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=RFBWD0MXEMM3ESEHAPAJMAPGOA4GF5NBWA15BU1CYSRLLWSZ&client_secret=M3LQZF1E0YG0EQJ0IUMDUFDQSGZN0RHNFXDH335O44CYPAIR&ll=43.70460773398059,-79.39715291165048&v=20180604&radius=500&limit=30'

#### Send get requests

In [35]:
results = requests.get(url).json()
#'There are {} around  restaurant.'.format(len(results['response']['groups'][0]['items']))

KeyError: 'repsonse'

In [39]:
items=results['response']['groups'][0]['items']

In [42]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = json_normalize(items) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in venues.columns if col.startswith('venue.location.')] + ['venue.id']
venues_filtered = venues.loc[:, filtered_columns]

# filter the category for each row
venues_filtered['venue.categories'] = venues_filtered.apply(get_category_type, axis=1)

# clean columns
venues_filtered.columns = [col.split('.')[-1] for col in venues_filtered.columns]

venues_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Zezafoun Syrian Cousine,Restaurant,NaN,CA,Toronto,Canada,NaN,216,"[Toronto ON M4S 2A4, Canada]","[{'label': 'display', 'lat': 43.702667, 'lng':...",43.702667,-79.397120,M4S 2A4,ON,5abe3be1d3cce81e4407f9e5
1,The Keg Steakhouse & Bar,Steakhouse,2201 Yonge St,CA,Toronto,Canada,at Eglinton Ave,121,"[2201 Yonge St (at Eglinton Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.70546405087482...",43.705464,-79.398085,M4S 2B2,ON,4ad772faf964a520a70a21e3
2,Starbucks,Coffee Shop,39 Eglinton Ave E,CA,Toronto,Canada,at Holly,260,"[39 Eglinton Ave E (at Holly), Toronto ON M4P ...","[{'label': 'display', 'lat': 43.70694, 'lng': ...",43.706940,-79.396908,M4P 1A2,ON,4ad9c161f964a520a11a21e3
3,Yonge Eglinton Square,Plaza,Yonge St.,CA,Toronto,Canada,at Eglinton Ave.,282,"[Yonge St. (at Eglinton Ave.), Toronto ON M4S,...","[{'label': 'display', 'lat': 43.70686372150796...",43.706864,-79.398751,M4S,ON,4c6c6c486af58cfae2ce8617
4,Five Doors North,Italian Restaurant,2088 Yonge St,CA,Toronto,Canada,NaN,265,"[2088 Yonge St, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.70223554183669...",43.702236,-79.397526,NaN,ON,4b6b758ff964a520270a2ce3
5,Loblaws,Supermarket,"101 Eglinton Ave E Toronto,Unit 1",CA,Toronto,Canada,NaN,360,"[101 Eglinton Ave E Toronto,Unit 1, Toronto ON...","[{'label': 'display', 'lat': 43.70741230863768...",43.707412,-79.394909,M4P 1H4,ON,5803b50738fa169524aca153
6,Balsamico,Italian Restaurant,2029 Yonge St.,CA,Toronto,Canada,NaN,345,"[2029 Yonge St., Toronto ON M4S 2A2, Canada]","[{'label': 'display', 'lat': 43.70150469426698...",43.701505,-79.397162,M4S 2A2,ON,4ae86fb7f964a52053af21e3
7,Little Sister,Indonesian Restaurant,2031 Yonge St,CA,Toronto,Canada,at Lola Rd,340,"[2031 Yonge St (at Lola Rd), Toronto ON M4S 2A...","[{'label': 'display', 'lat': 43.70155150459908...",43.701552,-79.397163,M4S 2A2,ON,53aca702498e3bbe34efc1bb
8,Noorden Food x Bar,Restaurant,2110 Yonge St.,CA,Toronto,Canada,NaN,203,"[2110 Yonge St., Toronto ON, Canada]","[{'label': 'display', 'lat': 43.70281430168457...",43.702814,-79.397645,NaN,ON,573e4cf6498e5c51f2319149
9,Fresh,Vegetarian / Vegan Restaurant,90 Eglinton Avenue East,CA,Toronto,Canada,Yonge & Eglinton,325,"[90 Eglinton Avenue East (Yonge & Eglinton), T...","[{'label': 'display', 'lat': 43.70732441045359...",43.707324,-79.395649,M4P 1A6,ON,521e0c6c04939a8ad55d93d3


### Explore the neighborhood

In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:

Toronto_venues = getNearbyVenues(names=Toronto_FSAs['Borough'],
                                   latitudes=Toronto_FSAs['Latitude'],
                                   longitudes=Toronto_FSAs['Longitude']
                                  )

Scarborough
[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

...

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5abe3be1d3cce81e4407f9e5', 'name': 'Zezafoun Syrian Cousine', 'location': {'lat': 43.702667, 'lng': -79.39712, 'labeledLatLngs': [{'label': 'display', 'lat': 43.702667, 'lng': -79.39712}], 'distance': 216, 'postalCode': 'M4S 2A4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON M4S 2A4', 'Canada']}, 'categories': [{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5abe3be1d3cce81e4407f9e5-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ad772faf964a52

In [52]:
Toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Zezafoun Syrian Cousine,43.702667,-79.397120,Restaurant
1,Scarborough,43.806686,-79.194353,The Keg Steakhouse & Bar,43.705464,-79.398085,Steakhouse
2,Scarborough,43.806686,-79.194353,Starbucks,43.706940,-79.396908,Coffee Shop
3,Scarborough,43.806686,-79.194353,Yonge Eglinton Square,43.706864,-79.398751,Plaza
4,Scarborough,43.806686,-79.194353,Five Doors North,43.702236,-79.397526,Italian Restaurant


In [54]:
Toronto_venues.to_csv(r"C:\Users\23165948\Documents\JupyterNotebooks\Coursera_Capstone\Toronto_venues.csv")